### config

In [1]:


from collections import defaultdict
from vif.baselines.models import VerEvaluation,VerEval_Features
from vif.baselines.verifiers_baseline.ver_baseline import TexVerBaseline
from datasets.formatting.formatting import LazyBatch
from PIL import Image
import sys
from datasets import load_dataset, Dataset, load_from_disk,concatenate_datasets
from loguru import logger

from vif.utils.renderer.tex_renderer import TexRenderer
import openai
import os
from google import genai
from google.genai import types as genTypes
from vif.baselines.verifiers_baseline import (
    FalconVerifier,
    TextVerifier,
    TextVisualVerifier,
    ViperGPTVerifier,
    VisualPropertiesVerifier,
    VisualVerifier,
    VisualCodeVerifier,
    PicoBananaVerifier,
    ClipSimVerifier
)
import datasets
from tqdm import tqdm
import json

logger.remove()
logger.add(tqdm.write)

/home/creux/Documents/AI/VIFagent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1

In [2]:

NUM_PROC = 1
client = openai.Client(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ.get("OPENROUTER_API_KEY"),
)

gclient = genai.Client(
        api_key=os.environ.get("GOOGLE_API_KEY"),
        http_options=genTypes.HttpOptions(api_version="v1alpha"),
    )

used_model = "qwen/qwen3-vl-8b-instruct"#"qwen/qwen3-vl-8b-thinking"

logger.configure(handlers=[{"sink": sys.stdout, "level": "WARNING"}])
ds = load_dataset("CharlyR/VeriTikz", "full", split="train")
ds = ds.select(range(45,46))
visual_verifier: TexVerBaseline = VisualCodeVerifier(
    model=used_model,
    temperature=0.5,
    client=client,
)

#def custom_get_code(vereval):
#    return open("notebooks/protos/code/d.py","r").read(),{}

#visual_verifier.get_code = custom_get_code


In [3]:
selected_row = ds[0]

In [4]:
selected_row

{'id': 'dispatch_slave_removed',
 'type': 'scientific',
 'instruction': 'Remove the first slave/replication instance, keep the other names the same.',
 'code': "\\documentclass[tikz,border=5]{standalone}\n\\usepackage[cmex10]{amsmath}\n\\usepackage[dvipsnames]{xcolor}\n\\usepackage[T1]{fontenc}\n\\usepackage{\n\tamsfonts,\n\tamssymb,\n\tamsthm,\n\tbbm,\n\tenumerate,\n\tfloat,\n\tdblfloatfix,\n\tdsfont,\n\tmathtools,\n\turl,\n\tpgf,\n\tpgfplots,\n\ttikz,\n\tsubcaption,\n}\n\\usepgflibrary{shapes}\n\\usetikzlibrary{\narrows,\nbackgrounds,\ncalc,\ncalendar,\nchains,\ndecorations,\ndecorations.pathmorphing,\nfit,\nmatrix,\nmindmap,\npetri,\npositioning,\nscopes,\nshadings,\nshadows,\nshapes,\nshapes.arrows,\nshapes.misc,\nshapes.symbols,\n}\n\n\\begin{document}\n\n\\begin{tikzpicture}\n[font=\\fontsize{6.75pt}{7.5}\\selectfont, line width=0.75pt, node distance=0mm, draw=black, >=stealth',\nserver/.style={cylinder, shape border rotate=90, aspect=0.35, minimum height=9mm, minimum width=10.5m

### Exec

In [6]:
test_ver_input = VerEvaluation(
    id=selected_row["id"],
    approach_name="test_approach_code",
    config_metadata={"test":"test"},
    initial_code=selected_row["code"],
    initial_image=selected_row["original_image"],
    initial_instruction=selected_row["instruction"],
    initial_solution=selected_row["solution"],
    initial_solution_image=selected_row["solution_image"],
    expected=selected_row["expected"],
    theoretical_perfect_image=selected_row["theoretical_perfect_image"],
)

res_input:VerEvaluation = visual_verifier.assess_customization(test_ver_input)

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [6]:
res_input.errors

{}

In [7]:
print(res_input.additional_metadata["generated_function"])

def execute_command(initial_image, modified_image):
    # Create ImagePatch objects for both images
    initial_patch = ImagePatch(initial_image)
    modified_patch = ImagePatch(modified_image)
    
    # Find cat in the initial image
    cats_initial = initial_patch.find("cat")
    if not cats_initial:
        return False  # No cat found in the original image
    
    # Find cat in the modified image
    cats_modified = modified_patch.find("cat")
    if not cats_modified:
        return False  # No cat found in the modified image
    
    # Get the first cat from each image (assuming one cat per image)
    cat_initial = cats_initial[0]
    cat_modified = cats_modified[0]
    
    # Check if the cat's nose is blue in the modified image
    # We'll use the llm_query to ask about the color of the nose
    nose_color = cat_modified.simple_query("What color is the nose?")
    
    # If the nose is not blue, return False
    if nose_color.lower() != "blue":
        return False
    
    # 

In [9]:
res_input.additional_metadata

{'generated_function': 'def execute_command(initial_image, modified_image):\n    # Create ImagePatch objects for both images\n    initial_patch = ImagePatch(initial_image)\n    modified_patch = ImagePatch(modified_image)\n    \n    # Find cat in the initial image\n    cats_initial = initial_patch.find("cat")\n    if not cats_initial:\n        return False  # No cat found in the original image\n    \n    # Find cat in the modified image\n    cats_modified = modified_patch.find("cat")\n    if not cats_modified:\n        return False  # No cat found in the modified image\n    \n    # Get the first cat from each image (assuming one cat per image)\n    cat_initial = cats_initial[0]\n    cat_modified = cats_modified[0]\n    \n    # Check if the cat\'s nose is blue in the modified image\n    # We\'ll use the llm_query to ask about the color of the nose\n    nose_color = cat_modified.simple_query("What color is the nose?")\n    \n    # If the nose is not blue, return False\n    if nose_color.l